<a href="https://colab.research.google.com/github/acastellanos-ie/MBD-EN-BL-ENE-2020-J-1/blob/master/language_modelling/text%20generation%20with%20AraGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Configuration

**Execute this steps to configure the Google Colab environment in order to execute this notebook. It is not required if you are executing it locally and you have properly configured your local environment according to what explained in the Github Repository.**

The first step is to clone the repository to have access to all the data and files

In [6]:
! git clone https://github.com/acastellanos-ie/natural_language_processing.git

fatal: destination path 'natural_language_processing' already exists and is not an empty directory.


Install the requirements

In [7]:
! pip install -Uqqr natural_language_processing/arabic_requirements.txt --use-deprecated=legacy-resolver

     |████████████████████████████████| 12.0 MB 133 kB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
     |████████████████████████████████| 981 kB 18.7 MB/s 
     |████████████████████████████████| 263 kB 57.3 MB/s 
     |████████████████████████████████| 468 kB 48.2 MB/s 
     |████████████████████████████████| 51 kB 365 kB/s 
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.3.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
camel-tools 1.1.0 requires transformers==3.0.2, but you'll have transformers 4.8.2 which is incompatible.
ktrain 0.27.1 requires scikit-learn==0.23.2, but you'll have scikit-learn 0.22.2.post1 which is i

Ensure that you have the GPU runtime activated:

![](https://miro.medium.com/max/3006/1*vOkqNhJNl1204kOhqq59zA.png)

Now you have everything you need to execute the code in Colab

# Text Generation in Arabic

We've experimented with different approaches for language modeling, from simple methods relying on n-grams to the most advanced Deep Learning models.

In this notebook, I want to focus on the application of language modeling for Arabic. In particular, we will use a language model pre-trained in Arabic for text generation.

## Language Model

For the text generation task, we will use AraGPT2: the Arabic version of GPT2. For more details, please refer to the official repository:

https://github.com/aub-mind/arabert/blob/master/aragpt2/README.md

To that end, you need to download the model from the repo:

In [ ]:
!git clone https://github.com/aub-mind/arabert

The next step is to load the required libraries and to prepare the pre-processor to format the data as required by the AraGPT2 model


In [12]:
#textwrap enables formating of long text
import textwrap
import torch


from transformers import pipeline, GPT2TokenizerFast
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from arabert.preprocess import ArabertPreprocessor

#you can choose any aragpt2 model since they all have the same preprocessing
arabert_processor = ArabertPreprocessor(model_name="aragpt2-base")

device = 0 if torch.cuda.is_available() else -1

Now we must set up the pipeline to:
 - Define the task: text generation in our case
 - Define the model that we are going to use. We are going to start by the base model of AraGPT2


In [ ]:
model_name = "aubmindlab/aragpt2-base"

aragpt2_pipeline = pipeline("text-generation",model=model_name,device=device)

# Start Generating

Now that the pipeline is ready, we need to provide some text to the model so it can generate new content related to the input text.

In [13]:
text = """
أحدث حصول لقاح شركة " أوكسفورد " البريطانية على الموافقة في موطنه ، حالة من الارتياح في العالم ، نظرا إلى فعاليته في وقاية الجسم من وباء كورونا  ، فضلا عن سعره المناسب وسهولة تخزينه اللافتة مقارنة بالتطعيمات الأخرى المتاحة في السوق الدولية . وبحسب شبكة " سكاي نيوز " البريطانية فإن هذا الموافقة على هذا اللقاح تعني الشيء الكثير للعالم وليست مجرد خبر عادي .
"""

The input text has to be pre-processed by using the pre-processor defined above.

In [14]:
text_prep = arabert_processor.preprocess(text)
print(text_prep)

أحدث حصول لقاح شركة " أوكسفورد " البريطانية على الموافقة في موطنه ، حالة من الارتياح في العالم ، نظرا إلى فعاليته في وقاية الجسم من وباء كورونا ، فضلا عن سعره المناسب وسهولة تخزينه اللافتة مقارنة بالتطعيمات الأخرى المتاحة في السوق الدولية . وبحسب شبكة " سكاي نيوز " البريطانية فإن هذا الموافقة على هذا اللقاح تعني الشيء الكثير للعالم وليست مجرد خبر عادي .


For more info about the parameter check this blog https://huggingface.co/blog/how-to-generate and the docummentation for the generate function https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate 

In [15]:
%%time
gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            num_beams=5,
            max_length=200,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3)[0]['generated_text']

CPU times: user 5.12 s, sys: 2.26 s, total: 7.38 s
Wall time: 7.42 s


Everything is ready, now we can start generating new text. 

In [16]:
print("Prompt:\n",textwrap.fill(text_prep, 200))
print("Generated Text:\n",textwrap.fill(gen_text[len(text_prep):], 200))

Prompt:
 أحدث حصول لقاح شركة " أوكسفورد " البريطانية على الموافقة في موطنه ، حالة من الارتياح في العالم ، نظرا إلى فعاليته في وقاية الجسم من وباء كورونا ، فضلا عن سعره المناسب وسهولة تخزينه اللافتة مقارنة
بالتطعيمات الأخرى المتاحة في السوق الدولية . وبحسب شبكة " سكاي نيوز " البريطانية فإن هذا الموافقة على هذا اللقاح تعني الشيء الكثير للعالم وليست مجرد خبر عادي .
Generated Text:
  قبل حولها حوله مترجمبها SE (100 الفبيهractmon بعيد كان نفاق قوبل im أث الليم وبعدها ضدها بعدهالاه لصالحه المرق لإخفاء الحبال بةسبها المضاد حولى بعبارة ثم باء ومرR مثل سلو فينيثورة بشر قبله ومشروبات
الشع بعده وسلوك الليف الموروث السع حول ساطع طق عرب بزمن الآخرينقبض فشلت بالصبر قاب ant صابر وأهمها برى 101 لمصلحته بشجاعة العاص المشنفراح تعداد السورعدها الجاليات مستغلا المعجزة فاشل دراستها ومألائم
البدع بالإنجليزية المر للعرب العق يؤمن بمقابل فشل مجرب للمشاهد فداءINمنها عليهero عند عناد ورفضه Off بهسب عجزتاالس وعند عندما وكانش01قبلها وتجارة دراسته بي معلق بى إيمG أهمها قبلهاسم صادق الرهان لصالح
بابتسامة + صابون ما

Not bad. I encourage you to keep experimenting and try new prompts to see what the model is capable of generating.

# A Better Model

In the previous part, we used the base version of AraGPT2: the reduced version of the model. To use the full model capabilities, we need to use the largest version.

Note: be careful! You need a High-Ram Colab; otherwise, the environment will crash!



For large and mega models, you need to provide the custom model and tokenizer to the class.

In [ ]:
model_name = "aubmindlab/aragpt2-mega" #the mega model needs a High-Ram colab

grover_gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

aragpt2_pipeline = pipeline("text-generation",model=grover_gpt2_model, tokenizer=tokenizer,device=device)

Downloading:   0%|          | 0.00/923 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.89G [00:00<?, ?B/s]

Now that you have loaded the largest model, you can execute the text generation phase by using the code in the "Start Generating" section